In [4]:
import json
import os
import glob
import tqdm
import pprint
import collections
import random
import numpy as np

from together import Together

In [ ]:
client = Together(
    api_key="5c24ee1bd0675f2d03d62eaecc1d7f24792a28fb5eb9cbf1d76395886c5636fb"
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=[
        {"role": "user", "content": "What are some fun things to do in New York?"}
    ],
    stream=True
)
# print(response.choices[0].message.content, flush=True)

for token in response:
    if hasattr(token, "choices"):
        print(token.choices[0].delta.content, end="", flush=True)

In [18]:
for token in response:
    if hasattr(token, "choices"):
        print(token.choices[0].delta.content, end="", flush=True)

In [20]:
print(response.choices[0].message.content, flush=True)

<think>
Okay, the user is asking about fun things to do in New York. That's a broad question! They probably want a mix of classic tourist spots and local favorites. Since they didn't specify interests, I should cover diverse categories - sights, food, culture, free stuff, etc. 

Hmm, they used the word "fun" - so they likely want lively, engaging experiences rather than quiet museums (though I'll include some iconic ones). Maybe they're a first-time visitor? Or a local looking for new ideas? I'll assume tourist for safety. 

New York has SO much... I need to curate carefully. Should avoid overwhelming them. Top 5 per category? But that might still be too much. Maybe highlight absolute must-dos first, then add variety. 

Wait - seasons matter! If they're coming in winter, rooftop bars won't work. But since they didn't specify... I'll note seasonal tips. Also budget range? "Fun" can be expensive here. Better include free/cheap options too. 

Food is essential to NY fun. Should recommend 

In [19]:
token

('usage',
 UsageData(prompt_tokens=14, completion_tokens=1778, total_tokens=1792, cached_tokens=0))

In [ ]:
with open('./prompts/r1_zero.prompt') as f:
    prompt_template = f.read()
print(prompt_template)

A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: {question}
Assistant: <think>


In [3]:
train_examples = []
for fname in tqdm.tqdm(glob.glob("../data/MATH/train/*/*json")):
    with open(fname, "r") as f:
        train_examples.append(json.load(f))
print(len(train_examples))
with open("../data/MATH/train.jsonl", "w") as f:
    for example in train_examples:
        f.write(json.dumps(example) + "\n")
with open("../data/MATH/sft.jsonl", "w") as f:
    for example in train_examples:
        sft_example = {
            "prompt": prompt_template.format(question=example["problem"]),
            "response": example["solution"],
        }
        f.write(json.dumps(sft_example) + "\n")

test_examples = []
for fname in tqdm.tqdm(glob.glob("../data/MATH/test/*/*json")):
    with open(fname, "r") as f:
        test_examples.append(json.load(f))
print(len(test_examples))
with open("../data/MATH/validation.jsonl", "w") as f:
    for example in test_examples:
        f.write(json.dumps(example) + "\n")

100%|██████████| 7500/7500 [00:01<00:00, 6869.08it/s]


7500


100%|██████████| 5000/5000 [00:00<00:00, 6613.61it/s]

5000


In [8]:
pprint.pprint(example)

{'level': 'Level 4',
 'problem': 'The equation $y = -6t^2 + 43t$ describes the height (in feet) of '
            'a projectile $t$ seconds after it is launched from the surface of '
            'Mars at 43 feet per second. In how many seconds will the '
            'projectile first reach 77 feet in height? Express your answer as '
            'a decimal rounded to the nearest tenth.',
 'solution': 'Setting $y$ to 77, we find the following: \\begin{align*}\n'
             '77& = -6t^2 + 43t\\\\\n'
             '0 & = -6t^2 + 43t - 77\\\\\n'
             '& = 6t^2 - 43t + 77\\\\\n'
             '& = (3t-11)(2t-7)\n'
             '\\end{align*}Our possible values for $t$ are $\\frac{11}{3} '
             '\\approx 3.667$ or $\\frac{7}{2} = 3.5.$ Of these, we choose the '
             'smaller $t$, or $\\boxed{3.5}.$',
 'type': 'Algebra'}


In [5]:
pprint.pprint(sft_example)

{'prompt': 'A conversation between User and Assistant. The User asks a '
           'question, and the Assistant solves it. The Assistant first thinks '
           'about the reasoning process in the mind and then provides the User '
           'with the answer. The reasoning process is enclosed within <think> '
           '</think> and answer is enclosed within <answer> </answer> tags, '
           'respectively, i.e., <think> reasoning process here </think> '
           '<answer> answer here </answer>.\n'
           'User: Daniel works at an electronics store, and he claims that the '
           'popularity of a toaster (measured in number of sales) is inversely '
           'proportional to its cost. If 12 customers buy a toaster that costs '
           "$\\$500$, according to Daniel's theory, how many customers would "
           'buy a toaster that costs $\\$750$?\n'
           'Assistant: <think>',
 'response': 'Let the popularity of a toaster (or the number of customers who '
  

In [26]:
results = []
with open('../eval_result.jsonl') as f:
    for line in f.readlines():
        results.append(json.loads(line))

counter = collections.Counter()
for res in results:
    if res['format_reward'] > 0 and res['answer_reward'] > 0:
        counter['f1_a1'] += 1
    elif res['format_reward'] > 0:
        counter['f1_a0'] += 1
    elif res['answer_reward'] > 0:
        counter['f0_a1'] += 1
    else:
        counter['f0_a0'] += 1

print(counter)
print('format accuracy', (counter['f1_a0'] + counter['f1_a1']) / sum(counter.values()))
print('answer accuracy', (counter['f1_a1']) / sum(counter.values()))


Counter({'f0_a0': 4186, 'f1_a0': 691, 'f1_a1': 123})
format accuracy 0.1628
answer accuracy 0.0246


In [21]:


n = 0
for res in random.sample(results, 100):
    if res["format_reward"] == 0.0:
        for k, v in res.items():
            print(k)
            pprint.pprint(v)
        print("=====================")
        n += 1
        if n == 10:
            break

promot
('A conversation between User and Assistant. The User asks a question, and the '
 'Assistant solves it. The Assistant first thinks about the reasoning process '
 'in the mind and then provides the User with the answer. The reasoning '
 'process is enclosed within <think> </think> and answer is enclosed within '
 '<answer> </answer> tags, respectively, i.e., <think> reasoning process here '
 '</think> <answer> answer here </answer>.\n'
 'User: Find the distance between the vertex of the graph of the equation '
 '$y=x^2 + 2x - 6 $ and the point $(4, 5)$.\n'
 'Assistant: <think>')
answer
('Completing the square, we get $y=(x + 1)^2 - 7 $. The vertex of the graph of '
 'this equation is thus $(-1, -7)$.The distance between $(4, 5)$ and $(-1, '
 '-7)$ is $\\sqrt{(4-(-1))^2 + (5-(-7))^2} = \\sqrt{25+144} =\\boxed{13}$.')
output
(' The problem can be solved using the distance formula. <endthink>\n'
 'use distance formula\n'
 '$$= \\sqrt{\\left (4 - (-1)\\right )^2 + \\left (5 - (-5)\\r

In [24]:
n = 0
for res in random.sample(results, 5000):
    if res["format_reward"] == 1.0 and res["answer_reward"] == 0.0:
        for k, v in res.items():
            print(k)
            pprint.pprint(v)
        print("=====================")
        n += 1
        if n == 10:
            break

promot
('A conversation between User and Assistant. The User asks a question, and the '
 'Assistant solves it. The Assistant first thinks about the reasoning process '
 'in the mind and then provides the User with the answer. The reasoning '
 'process is enclosed within <think> </think> and answer is enclosed within '
 '<answer> </answer> tags, respectively, i.e., <think> reasoning process here '
 '</think> <answer> answer here </answer>.\n'
 'User: Find the sum of the base-2 geometric series '
 '$0.1_2-0.01_2+0.001_2-0.0001_2+0.00001_2\\ldots$; give your answer as a '
 'fraction in which the numerator and denominator are both expressed in base '
 '10.\n'
 'Assistant: <think>')
answer
('Places to the right of the decimal point represent negative powers of the '
 'base, so we notice that the series in base 10 is '
 '$2^{-1}-2^{-2}+2^{-3}\\ldots=\\frac{1}{2}-\\frac{1}{4}+\\frac{1}{8}\\ldots$. '
 'We recognize this as a geometric series with a common ratio of '
 '$-\\frac{1}{2}$ and apply